**Demo for `teneva.als`**

---

This module contains the function "als" which computes the TT-approximation for the tensor by TT-ALS algorithm, using given random samples.

## Loading and importing modules

In [4]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)


In [5]:
# from qaoa_utils import *
import time
import matplotlib.pyplot as plt
import networkx as nx
from functools import partial
from qokit.qaoa_objective_maxcut import get_qaoa_maxcut_objective
from qokit.maxcut import maxcut_obj, get_adjacency_matrix
from qokit.utils import precompute_energies


## Function `als`

Build TT-tensor by TT-ALS method using given random tensor samples.

In [6]:
p = 4
d = int(p*2)                           # Dimension of the function
a = np.ones(d)*-np.pi   # Lower bounds for spatial grid
b = np.ones(d)*np.pi   # Upper bounds for spatial grid
n = np.ones(d)*10   # Shape of the tensor


In [7]:
N = 6
seed = 1
G = nx.random_regular_graph(3,N,seed=seed)

# precompute energies to speed up computation
obj = partial(maxcut_obj, w=get_adjacency_matrix(G))
precomputed_energies = precompute_energies(obj, N)
optimal_cut = np.max(precomputed_energies)
func0 = get_qaoa_maxcut_objective(N, p, precomputed_cuts=precomputed_energies, parameterization='theta')

# G = nx.random_regular_graph(d=3, n=6, seed=12345)
# func0 = get_black_box_objective_multiple(G, p=p, n_trials=1024, repeat=1, sv=True) 


We set the target function (the function takes as input a set of tensor multi-indices I of the shape [samples, dimension], which are transformed into points X of a uniform spatial grid using the function "ind_to_poi"):

In [8]:
def func(I):
    """Schaffer function."""
    X = teneva.ind_to_poi(I, a, b, n)
    # return func0(X)
    n_sample = I.shape[0]
    y = np.zeros(n_sample)
    for i in range(n_sample):
        y[i] = func0(X[i])
    return y


We prepare train data from the LHS random distribution:

In [9]:
m    = 4.E+6   # Number of calls to target function
I_trn = teneva.sample_lhs(n, m, seed=42) 
y_trn = func(I_trn)


We prepare test data from the random tensor multi-indices:

In [10]:
m_tst = 1.E+6
I_tst = teneva.sample_rand(n, m_tst, seed=2023) 
y_tst = func(I_tst)


In [11]:
# nswp = 10     # Sweep number for ALS iterations
# r    = 5      # TT-rank of the initial random tensor

t = tpc()
Y = teneva.anova(I_trn, y_trn, r=10, order=2)
Y = teneva.als(I_trn, y_trn, Y, lamb=0.001, log=True)
# Y = teneva.als(I_trn, y_trn, Y, nswp=10, r=r, e_adap=1.E-2, lamb=0.1, log=True)
t = tpc() - t

print(f'Build time : {t:-10.2f}')


# pre | time:      0.836 | rank:  10.0 | 
#   1 | time:    230.152 | rank:  10.0 | e: 9.6e-02 | 
#   2 | time:    419.216 | rank:  10.0 | e: 6.5e-02 | 
#   3 | time:    609.542 | rank:  10.0 | e: 3.2e-02 | 
#   4 | time:    798.344 | rank:  10.0 | e: 2.5e-02 | 
#   5 | time:    978.979 | rank:  10.0 | e: 1.2e-02 | 
#   6 | time:   1155.125 | rank:  10.0 | e: 1.1e-02 | 
#   7 | time:   1334.749 | rank:  10.0 | e: 9.0e-03 | 
#   8 | time:   1499.958 | rank:  10.0 | e: 8.0e-03 | 
#   9 | time:   1636.313 | rank:  10.0 | e: 7.3e-03 | 
#  10 | time:   1772.514 | rank:  10.0 | e: 5.9e-03 | 
#  11 | time:   1915.857 | rank:  10.0 | e: 4.3e-03 | 
#  12 | time:   2052.892 | rank:  10.0 | e: 3.4e-03 | 
#  13 | time:   2188.869 | rank:  10.0 | e: 3.0e-03 | 
#  14 | time:   2324.900 | rank:  10.0 | e: 2.9e-03 | 
#  15 | time:   2461.646 | rank:  10.0 | e: 2.9e-03 | 
#  16 | time:   2598.239 | rank:  10.0 | e: 2.8e-03 | 
#  17 | time:   2734.597 | rank:  10.0 | e: 3.1e-03 | 
#  18 | time:   2870.60

And now we can check the result:

In [12]:
# Compute approximation in train points:
y_our = teneva.get_many(Y, I_trn)

# Accuracy of the result for train points:
e_trn = np.linalg.norm(y_our - y_trn)          
e_trn /= np.linalg.norm(y_trn)

# Compute approximation in test points:
y_our = teneva.get_many(Y, I_tst)

# Accuracy of the result for test points:
e_tst = np.linalg.norm(y_our - y_tst)          
e_tst /= np.linalg.norm(y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

y_tst.sort()
y_our.sort()
print(f'NRMSE on test: {np.linalg.norm(y_our - y_tst)/np.sqrt(m_tst)/(y_tst[int(m_tst/4*3)] - y_tst[int(m_tst/4)])}')


Error on train :   1.58e-01
Error on test  :   1.59e-01
NRMSE on test: 0.303703223951617


We can also set a validation data set and specify as a stop criterion the accuracy of the TT-approximation on this data (and we can also present the logs):

In [13]:
I_vld = teneva.sample_rand(n, 1.E+3, seed=99) 
y_vld = func(I_vld)


In [14]:
t = tpc()
Y = teneva.anova(I_trn, y_trn, r)
Y = teneva.als(I_trn, y_trn, Y, nswp, I_vld=I_vld, y_vld=y_vld, e_vld=1.E-2, log=True)
t = tpc() - t

print(f'\nBuild time     : {t:-10.2f}')


NameError: name 'r' is not defined

We can use helper functions to present the resulting accuracy:

In [ ]:
print(f'Error on train : {teneva.accuracy_on_data(Y, I_trn, y_trn):-10.2e}')
print(f'Error on valid.: {teneva.accuracy_on_data(Y, I_vld, y_vld):-10.2e}')
print(f'Error on test  : {teneva.accuracy_on_data(Y, I_tst, y_tst):-10.2e}')


Error on train :   1.27e-03
Error on valid.:   5.01e-02
Error on test  :   4.95e-02


We may also set the value of relative rate of solution change to stop the iterations:

In [ ]:
t = tpc()
Y = teneva.anova(I_trn, y_trn, r)
Y = teneva.als(I_trn, y_trn, Y, e=1.E-3, I_vld=I_vld, y_vld=y_vld, log=True)
t = tpc() - t

print(f'\nBuild time     : {t:-10.2f}')


# pre | time:      0.002 | rank:   5.0 | e_vld: 2.1e-01 | 
#   1 | time:      0.049 | rank:   5.0 | e_vld: 1.1e-01 | e: 1.9e-01 | 
#   2 | time:      0.087 | rank:   5.0 | e_vld: 2.2e-02 | e: 1.1e-01 | 
#   3 | time:      0.124 | rank:   5.0 | e_vld: 1.0e-02 | e: 1.6e-02 | 
#   4 | time:      0.158 | rank:   5.0 | e_vld: 6.5e-03 | e: 4.1e-03 | 
#   5 | time:      0.197 | rank:   5.0 | e_vld: 4.6e-03 | e: 2.3e-03 | 
#   6 | time:      0.239 | rank:   5.0 | e_vld: 3.7e-03 | e: 1.2e-03 | 
#   7 | time:      0.278 | rank:   5.0 | e_vld: 3.1e-03 | e: 7.7e-04 | stop: e | 

Build time     :       0.28


In [ ]:
print(f'Error on train : {teneva.accuracy_on_data(Y, I_trn, y_trn):-10.2e}')
print(f'Error on valid.: {teneva.accuracy_on_data(Y, I_vld, y_vld):-10.2e}')
print(f'Error on test  : {teneva.accuracy_on_data(Y, I_tst, y_tst):-10.2e}')


Error on train :   2.63e-03
Error on valid.:   3.11e-03
Error on test  :   2.94e-03


We may also pass callback function (it will be called after every sweep):

In [ ]:
def cb(Y, info, opts):
    e = teneva.accuracy(Y, opts['Yold'])
    print(f'Callback : e={e:-7.1e}')
    if info['nswp'] == 5:
        # Stop the algorithm's work
        return True


In [ ]:
t = tpc()
Y = teneva.anova(I_trn, y_trn, r)
Y = teneva.als(I_trn, y_trn, Y, e=1.E-10, cb=cb, log=True)
t = tpc() - t

print(f'\nBuild time     : {t:-10.2f}')


# pre | time:      0.001 | rank:   5.0 | 
Callback : e=1.9e-01
#   1 | time:      0.047 | rank:   5.0 | e: 1.9e-01 | 
Callback : e=1.1e-01
#   2 | time:      0.082 | rank:   5.0 | e: 1.1e-01 | 
Callback : e=1.8e-02
#   3 | time:      0.122 | rank:   5.0 | e: 1.8e-02 | 
Callback : e=2.3e-03
#   4 | time:      0.162 | rank:   5.0 | e: 2.3e-03 | 
Callback : e=8.8e-04
#   5 | time:      0.202 | rank:   5.0 | e: 8.8e-04 | stop: cb | 

Build time     :       0.21


We can also use rank-adaptive version of the TT-ALS method (note that result is very sensitive to "r" and "lamb" parameter values):

In [ ]:
t = tpc()
Y = teneva.anova(I_trn, y_trn, r=2)
Y = teneva.als(I_trn, y_trn, Y, nswp=5,
    I_vld=I_vld, y_vld=y_vld, r=5, e_adap=1.E-2, lamb=0.00001, log=True)
t = tpc() - t

print(f'\nBuild time     : {t:-10.2f}')


# pre | time:      0.002 | rank:   2.0 | e_vld: 2.1e-01 | 
#   1 | time:      0.180 | rank:   5.0 | e_vld: 2.1e-02 | e: 2.2e-01 | 
#   2 | time:      0.353 | rank:   3.9 | e_vld: 6.9e-03 | e: 2.0e-02 | 
#   3 | time:      0.514 | rank:   3.9 | e_vld: 6.5e-03 | e: 2.6e-03 | 
#   4 | time:      0.675 | rank:   4.2 | e_vld: 7.6e-03 | e: 6.3e-03 | 
#   5 | time:      0.840 | rank:   3.9 | e_vld: 6.6e-03 | e: 6.3e-03 | stop: nswp | 

Build time     :       0.85


In [ ]:
print(f'Error on train : {teneva.accuracy_on_data(Y, I_trn, y_trn):-10.2e}')
print(f'Error on valid.: {teneva.accuracy_on_data(Y, I_vld, y_vld):-10.2e}')
print(f'Error on test  : {teneva.accuracy_on_data(Y, I_tst, y_tst):-10.2e}')


Error on train :   6.54e-03
Error on valid.:   6.58e-03
Error on test  :   7.22e-03


We can also specify weights for elements of the training dataset. In the following example, we set increased weights for the first 1000 points from the set and expect that the accuracy of the result on them will be higher than on the rest:

In [ ]:
m = len(I_trn)
dm = 1000
I_trn1, y_trn1 = I_trn[:dm], y_trn[:dm]
I_trn2, y_trn2 = I_trn[dm:], y_trn[dm:]

w = np.ones(m)
w[:dm] = 100.


In [ ]:
t = tpc()
Y = teneva.anova(I_trn, y_trn, r)
Y = teneva.als(I_trn, y_trn, Y, w=w)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')


Build time     :       1.96


In [ ]:
print(f'Error full data : {teneva.accuracy_on_data(Y, I_trn, y_trn):-10.2e}')
print(f'Error for part1 : {teneva.accuracy_on_data(Y, I_trn1, y_trn1):-10.2e}')
print(f'Error for part2 : {teneva.accuracy_on_data(Y, I_trn2, y_trn2):-10.2e}')


Error full data :   9.89e-03
Error for part1 :   2.68e-03
Error for part2 :   1.04e-02


---